In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 🎯 Theoretical minimum riding time
t_maxspeed = 48  # Update if needed

# 🎨 Base color scheme
base_olive = "#4F7942"        # Darker olive for Alighted
light_olive = "#E07B39"       # Lighter olive for In-Transit

def extract_relative_delays_per_case(base_dir, case_densities, case_labels, num_trials=50):
    cases = sorted([
        d for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d)) and d.startswith('Case')
    ])

    if len(cases) != len(case_labels):
        raise ValueError("Mismatch between case folders and labels!")

    data = {
        "Case": [],
        "Relative Delay": [],
        "Kappa": [],
        "Rate": [],
        "Status": []
    }

    for case_dir, label in zip(cases, case_labels):
        print(f"Processing Case: {label}, Folder: {case_dir}")
        case_path = os.path.join(base_dir, case_dir)

        for kappa_dir in os.listdir(case_path):
            if not kappa_dir.startswith('Kappa_'):
                continue
            kappa_value = kappa_dir.split('_')[1]
            print(f"  ↪ Kappa: {kappa_value}")
            
            try:
                target_density = case_densities[label][kappa_value]
            except KeyError:
                print(f"  ⚠️ Skipping: No density for ({label}, {kappa_value})")
                continue

            kappa_path = os.path.join(case_path, kappa_dir)

            for density_dir in os.listdir(kappa_path):
                if density_dir != f'Density_{target_density}':
                    continue
                density_path = os.path.join(kappa_path, density_dir)
                print(f"    ✔ Density matched: {target_density}")

                for rate_dir in os.listdir(density_path):
                    if not rate_dir.startswith('PassengerRate_'):
                        continue
                    rate_value = rate_dir.split('_')[1]
                    rate_path = os.path.join(density_path, rate_dir, 'PassengerData')
                    if not os.path.exists(rate_path):
                        continue

                    print(f"      ⏫ Rate: {rate_value}")

                    for trial in range(1, num_trials + 1):
                        trial_file = f'Trial_{trial}_D{target_density}_K{kappa_value}_R{rate_value}_S20.csv'
                        file_path = os.path.join(rate_path, trial_file)

                        if os.path.exists(file_path):
                            df = pd.read_csv(file_path)
                            if {'Spawning Time', 'Riding Time', 'Riding Status'}.issubset(df.columns):
                                df = df[df['Spawning Time'] > 6999]
                                df = df[df['Riding Status'].isin(['In-Transit', 'Alighted'])]

                                for _, row in df.iterrows():
                                    if pd.notna(row['Riding Time']):
                                        delay = row['Riding Time'] - t_maxspeed
                                        rel_delay = delay / t_maxspeed if t_maxspeed > 0 else 0
                                        data["Case"].append(label)
                                        data["Relative Delay"].append(rel_delay)
                                        data["Kappa"].append(kappa_value)
                                        data["Rate"].append(rate_value)
                                        data["Status"].append(row['Riding Status'])

    return pd.DataFrame(data)

def plot_relative_delays(df, case_labels):
    unique_kappas = sorted(df['Kappa'].unique(), key=lambda x: float(x))
    unique_rates = sorted(df['Rate'].unique(), key=lambda x: float(x))

    for kappa_value in unique_kappas:
        for rate_value in unique_rates:
            filtered_df = df[
                (df['Kappa'] == kappa_value) &
                (df['Rate'] == rate_value)
            ]

            if filtered_df.empty:
                continue

            print(f"\n📊 Plotting: Kappa={kappa_value}, Rate={rate_value}")

            plt.figure(figsize=(12, 6))
            sns.set_theme(style="ticks", context="talk", font_scale=1.2)

            # Draw boxplots per Status
            sns.boxplot(
                data=filtered_df,
                x="Relative Delay",
                y="Case",
                hue="Status",
                hue_order=["Alighted", "In-Transit"],
                palette={
                    "Alighted": base_olive,
                    "In-Transit": light_olive
                },
                showfliers=False
            )

            plt.xlabel(r"$t_{\mathrm{delay, relative}}$", fontsize=35)
            plt.ylabel("", fontsize=45)
            plt.xticks(fontsize=35)
            plt.yticks(fontsize=45)
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.legend(fontsize=15, loc='upper right', bbox_to_anchor=(1, 0.97), ncol=1,frameon=True)
            sns.despine()

            for spine in plt.gca().spines.values():
                spine.set_visible(True)

            plt.tight_layout()
            fname = f"Stacked_Relative_Travel_Delay_Kappa_{kappa_value}_Rate_{rate_value}_targetted_0.2.png"
            plt.savefig(fname, dpi=200)
            plt.show()

# ============================
# 🛠️ Set your parameters here
# ============================
base_dir = os.path.abspath(os.path.join(os.getcwd(), '../../..', 'Load Anywhere Output'))

case_densities = {
    "OO": {"0": 0.2, "0.2": 0.2, "0.4": 0.2},
    r"$O\tilde{T}$": {"0": 0.2, "0.2": 0.2, "0.4": 0.2},
    r"$\tilde{J}O$": {"0": 0.2, "0.2": 0.2, "0.4": 0.2},
    r"$\tilde{J}\tilde{T}$": {"0": 0.2, "0.2": 0.2, "0.4": 0.2}
}

case_labels = list(case_densities.keys())

# 🔄 Run full pipeline
df_delay = extract_relative_delays_per_case(base_dir, case_densities, case_labels, num_trials=50)
plot_relative_delays(df_delay, case_labels)
